In [1]:
import pandas as pd

In [2]:
read_file = pd.read_csv("Embedding_Data.csv")

In [3]:
read_file.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,ML_Algo
0,0,-0.051891,-0.015976,-0.061629,0.046399,-0.018213,0.061907,0.018400,0.033098,-0.013741,...,0.036289,-0.027079,0.074891,-0.059269,0.017839,0.061928,-0.000457,-0.074404,-0.073431,K_Means
1,1,-0.031023,-0.007619,-0.072375,0.048698,-0.015002,0.070546,0.015014,-0.010198,-0.009262,...,0.017093,0.025700,0.054007,-0.017427,-0.033587,-0.042488,-0.019034,-0.081012,-0.046534,LSTM
2,2,0.008872,-0.030645,-0.064635,0.073782,-0.033790,0.074225,-0.033651,0.033711,-0.008916,...,0.023938,-0.037015,0.073519,-0.001105,0.025220,0.026691,0.003299,-0.063534,-0.069485,DBSCAN
3,3,-0.018570,-0.065392,-0.060226,0.068608,-0.010188,0.079610,0.012476,0.017613,0.026185,...,0.018482,-0.017448,0.072176,-0.046835,-0.035205,-0.042301,-0.034488,-0.079061,-0.064798,ARIMA
4,4,-0.051853,-0.047139,-0.069644,0.039946,-0.041594,0.076792,0.018718,0.014783,0.039036,...,0.028892,-0.004643,0.053501,-0.031011,-0.046399,-0.050395,0.012029,-0.084268,-0.056427,XgBoost


In [4]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
import logging
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

In [5]:
class FeatureSet(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, Y=None):
        return self

    def transform(self, X):
        return X[:,0:512]

In [16]:
class LabelEncoderTransform(BaseEstimator,TransformerMixin):

    def __init__(self):
        self.le = LabelEncoder()

    def fit(self, X, Y=None):
        self.le.fit(X[:,-1])
        print(self.le.classes_)
        return self

    def transform(self, X):
        return self.le.transform(X[:,-1]).reshape(132,1)

In [17]:
class XGBoost_Predictor(BaseEstimator,TransformerMixin):
    
    def __init__(self):
        self.xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
        
    def fit(self, X, Y=None):
        self.xgb_model.fit(X[:,0:512], X[:,-1])
        return self
        
    def transform(self, X, Y=None):
        return self.xgb_model, self.xgb_model.predict(X[:,0:512])

In [18]:
combine_feat_label = FeatureUnion(transformer_list=
        [
            ("Feature_Combine", FeatureSet()),
            ("Label_Encoder", LabelEncoderTransform())
        ]
    )

In [19]:
model_train = Pipeline(steps=
                       [
                           ("Feature_Union", combine_feat_label),
                           ("XGBoost_Predictor", XGBoost_Predictor())
                       ]
                      )

In [20]:
get_model, get_transform_data = model_train.fit_transform(read_file.to_numpy())

['ARIMA' 'AssociativeLearning' 'DBSCAN' 'K_Means' 'LSTM' 'XgBoost']


In [23]:
import tensorflow as tf
import tensorflow_hub as hub
def gen_test_data_predict(xg_model):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)
    label_dict = {0:'ARIMA', 1:'AssociativeLearning', 2:'DBSCAN', 3:'K_Means', 4:'LSTM', 5:'XgBoost'}
    print("module %s loaded" % module_url)
    result_dict = {}
    def embed(input_sent):
        return model(input_sent)
    test_data_load = pd.read_csv("Test_Data.csv")
    use_case_list = list(test_data_load['Use_Cases'])
    sentence_embeddings = pd.DataFrame(embed(use_case_list).numpy(),columns=list(range(0, 512)))
    get_labels = test_data_load[['ML_Algo']]
    concat_feat_label = pd.concat([sentence_embeddings, get_labels], axis = 1)
    print(f"Total Number of generated Embeddings are {len(sentence_embeddings)}")
    print(f"Total Number of Use-Case {len(test_data_load)}")
    print(f"Length of single embedding {len(sentence_embeddings[0])}")
    get_test_data_predict = xg_model.predict(concat_feat_label.to_numpy()[:,0:512])
    for uc, predict_label in zip(use_case_list, get_test_data_predict):
        result_dict[uc] = label_dict.get(predict_label)
    return result_dict
    

In [24]:
gen_test_data_predict(get_model)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


2023-10-05 01:24:11.533701: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]


Total Number of generated Embeddings are 2
Total Number of Use-Case 2
Length of single embedding 2


{'Predict the growth of product Nike Sports shoes within next year': 'K_Means',
 'Generate products similar to the Adidas winter wear': 'AssociativeLearning'}